In [1]:
import numpy as np
import ctypes

In [2]:
%%time
r1,c1,r2,c2 = 20,25,25,30
matrix1 = np.random.randn(r1,c1)
matrix2 = np.random.randn(r2,c2)
matrix_result = np.zeros((r1,c2))

Wall time: 0 ns


# 1. Using C language

In [3]:
import ctypes

In [4]:
def c_matmul(matrix1,matrix2,r1,c1,r2,c2):
    # 加载.dll or .so 文件
    lib = ctypes.cdll.LoadLibrary('lib/matmul.so')  
    # 定义返回的数据类型
    lib.matmul.restype = ctypes.POINTER(ctypes.c_double)
    # 定义输入的数据类型和相应的数值
    m1 = (ctypes.c_double*(r1*c1))(*list(matrix1.reshape(-1)))
    m2 = (ctypes.c_double*(r2*c2))(*list(matrix2.reshape(-1)))
    r1_c = ctypes.c_int(r1)
    r2_c = ctypes.c_int(r2)
    c1_c = ctypes.c_int(c1)
    c2_c = ctypes.c_int(c2)
    # 运行c函数, 引用的方式运行！
    m = lib.matmul(ctypes.byref(m1),ctypes.byref(m2),r1_c,c1_c,r2_c,c2_c)
    d = np.array([m[i] for i in range(r1*c2)]).reshape(r1,c2)
    return d,m

In [5]:
%%time
for i in range(r1):
    for j in range(c2):
        a = 0
        for k in range(c1):
            a += matrix1[i][k]*matrix2[k][j]
        matrix_result[i][j] = a

Wall time: 35.9 ms


In [6]:
%%time
m1 = c_matmul(matrix1,matrix2,r1,c1,r2,c2)

Wall time: 6.95 ms


In [7]:
%%time
m2 = np.matmul(matrix1,matrix2)

Wall time: 986 µs


In [8]:
def c_memory_leak(matrix1,size):
    # 加载.dll or .so 文件
    lib = ctypes.cdll.LoadLibrary('lib/memory leak.so')  
    # 定义返回的数据类型
    lib.memory_leak.restype = ctypes.POINTER(ctypes.c_double)
    # 定义输入的数据类型和相应的数值
    m1 = (ctypes.c_double*size)(*list(matrix1.reshape(-1)))

    size_c = ctypes.c_int(size)

    # 运行c函数, 引用的方式运行！
    m = lib.memory_leak(ctypes.byref(m1),size_c)
    d = np.array([m[i] for i in range(size)])
    return d,m